# CNN과 Transfer Learning으로 cats vs dogs 문제를 풀어봅시다

In [ ]:
pip install tensorflow_datasets

In [ ]:
conda install -c anaconda tensorflow-datasets

In [ ]:
 pip install tf-nightly

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import VGG16

from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

C:\Users\ebiz\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ebiz\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ebiz\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ebiz\AppData\Local\Continuum

ImportError: cannot import name 'trace' from 'tensorflow.python.profiler' (C:\Users\ebiz\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\profiler\__init__.py)

In [ ]:
uninstall TensorFlow and reinstall it (tensorflow==1.0.1 for me)

In [ ]:
pip install protobuf==3.6

In [4]:
tf.autograph.experimental.do_not_convert(
    func=None
)

AttributeError: module 'tensorflow._api.v1.autograph.experimental' has no attribute 'do_not_convert'

In [3]:
tf.compat.v2.autograph.experimental.do_not_convert

AttributeError: module 'tensorflow._api.v1.compat.v2.autograph.experimental' has no attribute 'do_not_convert'

In [2]:
# coding=utf-8
# Copyright 2022 The TensorFlow Datasets Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Cats vs Dogs dataset."""

import re

from absl import logging
import tensorflow as tf
import tensorflow_datasets.public_api as tfds

_CITATION = """\
@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
author = {Elson, Jeremy and Douceur, John (JD) and Howell, Jon and Saul, Jared},
title = {Asirra: A CAPTCHA that Exploits Interest-Aligned Manual Image Categorization},
booktitle = {Proceedings of 14th ACM Conference on Computer and Communications Security (CCS)},
year = {2007},
month = {October},
publisher = {Association for Computing Machinery, Inc.},
url = {https://www.microsoft.com/en-us/research/publication/asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization/},
edition = {Proceedings of 14th ACM Conference on Computer and Communications Security (CCS)},
}
"""

_URL = ("https://download.microsoft.com/download/3/E/1/3E1C3F21-"
        "ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip") 
#https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
_NUM_CORRUPT_IMAGES = 1738
_DESCRIPTION = (("A large set of images of cats and dogs. "
                 "There are %d corrupted images that are dropped.") %
                _NUM_CORRUPT_IMAGES)

_NAME_RE = re.compile(r"^PetImages[\\/](Cat|Dog)[\\/]\d+\.jpg$")


class CatsVsDogs(tfds.core.GeneratorBasedBuilder):
  """Cats vs Dogs."""

  VERSION = tfds.core.Version("4.0.0")
  RELEASE_NOTES = {
      "4.0.0": "New split API (https://tensorflow.org/datasets/splits)",
  }

  def _info(self):
    return tfds.core.DatasetInfo(
        builder=self,
        description=_DESCRIPTION,
        features=tfds.features.FeaturesDict({
            "image": tfds.features.Image(),
            "image/filename": tfds.features.Text(),  # eg 'PetImages/Dog/0.jpg'
            "label": tfds.features.ClassLabel(names=["cat", "dog"]),
        }),
        supervised_keys=("image", "label"),
        homepage="https://www.microsoft.com/en-us/download/details.aspx?id=54765",
        citation=_CITATION,
    )

  def _split_generators(self, dl_manager):
    path = dl_manager.download(_URL)

    # There is no predefined train/val/test split for this dataset.
    return [
        tfds.core.SplitGenerator(
            name=tfds.Split.TRAIN,
            gen_kwargs={
                "archive": dl_manager.iter_archive(path),
            }),
    ]

  def _generate_examples(self, archive):
    """Generate Cats vs Dogs images and labels given a directory path."""
    num_skipped = 0
    for fname, fobj in archive:
      res = _NAME_RE.match(fname)
      if not res:  # README file, ...
        continue
      label = res.group(1).lower()
      if tf.compat.as_bytes("JFIF") not in fobj.peek(10):
        num_skipped += 1
        continue
      record = {
          "image": fobj,
          "image/filename": fname,
          "label": label,
      }
      yield fname, record

    if num_skipped != _NUM_CORRUPT_IMAGES:
      raise ValueError("Expected %d corrupt images, but found %d" %
                       (_NUM_CORRUPT_IMAGES, num_skipped))
    logging.warning("%d images were corrupted and were skipped", num_skipped)

AttributeError: module 'tensorflow._api.v1.autograph.experimental' has no attribute 'do_not_convert'

# 1. 데이터셋 준비
### train data 80% test data 20%
### `BATCH_SIZE`와 `EPOCHS`(시간상 5) 를 정의합니다. 나중에 따로 EPOCHS = 20으로 하고 돌려보시길 바랍니다.

In [ ]:
BATCH_SIZE = 64
EPOCHS = 50   #open("u.item", encoding="utf-8")
dataset_name = 'cats_vs_dogs'
(train_dataset, validation_dataset), info = tfds.load(name=dataset_name, split=('train[:80%]', 'train[-20%:]'), with_info=True)
# open("u.item", encoding="utf-8")

# 2. 데이터 프리프로세싱

### 이미지에 대한 Normalization 합니다.
### 이미지는 0 ~ 255 까지 integer. tf.cast는 float로 바꾸고 255로 나누면 0~1까지 소수로 변경됨.  
### 이미지는 모두 같은 size로 만들어야 tensorflow에 입력이 가능함

In [ ]:
def normalize(images):
    # 0~1 사이의 값으로 Normalize 합니다.
    img, lbl = tf.cast(images['image'], tf.float32) / 255.0, images['label']
    # 300 X 300 사이즈로 resize 합니다.
    img = tf.image.resize(img, size=(300, 300))
    return img, lbl

`normalize`를 수행합니다.

`Training Set`와 `Validation Set`을 만들고 **BATCH_SIZE**에 따라 mini batch를 만듭니다.

In [ ]:
train_data = train_dataset.map(normalize).batch(BATCH_SIZE).repeat().prefetch(1)
valid_data = validation_dataset.map(normalize).batch(BATCH_SIZE).repeat().prefetch(1)

`steps_per_epoch`과 `validation_steps`를 정의하여 train 합니다.

In [ ]:
#steps_per_epoch= int(len(list(train_data)) * 0.8) // BATCH_SIZE + 1
#validation_steps= int(len(list(validation_data)) * 0.2) // BATCH_SIZE + 1
steps_per_epoch=582
validation_steps=146

# 4. 모델 설계하기(CNN) 

In [ ]:
model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3), padding='same'),
    MaxPooling2D(2, 2), 
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2), 
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2), 
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2), 
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2), 
    Flatten(), 
    Dense(64, activation='relu'), 
    Dropout(0.2),
    Dense(32, activation='relu'), 
    Dense(1, activation='sigmoid')
])
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_cnn.summary()

# 6. 모델 모니터링하기

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

checkpoint_path = 'checkpoint.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path, 
        save_best_only=True, 
        save_weights_only=True, 
        monitor='val_loss', 
        verbose=1)

# 5. 모델 학습하기

In [ ]:
cnn_history = model_cnn.fit(train_data, 
                            validation_data=(valid_data),
                            steps_per_epoch=steps_per_epoch,
                            validation_steps=validation_steps,
                            epochs=2, 
                            callbacks=[checkpoint, early_stopping])
model_cnn.load_weights(checkpoint_path)                      
model_cnn.save("cats-dogs-cnn3.h5")      

In [ ]:
model_cnn.load_weights(checkpoint_path)                      
model_cnn.save("cats-dogs-cnn1.h5")     

In [ ]:
cnn_history = model_cnn.fit(train_data, 
                            validation_data=(valid_data),
                            steps_per_epoch=steps_per_epoch,
                            validation_steps=validation_steps,
                            epochs=2, 
                            callbacks=[checkpoint, early_stopping])
model_cnn.load_weights(checkpoint_path)                      
model_cnn.save("cats-dogs-cnn3.h5")      

In [ ]:
model_cnn.load_weights(checkpoint_path)                      
model_cnn.save("cats-dogs-cnn3.h5")   

## Transfer Learning (VGG 16)

**VGG-16**은 ImageNet 데이터베이스의 1백만 개가 넘는 영상에 대해 훈련된 컨벌루션 신경망입니다. 이 네트워크에는 16개의 계층이 있으며, 영상을 키보드, 마우스, 연필, 각종 동물 등 1,000가지 사물 범주로 분류할 수 있습니다. 그 결과 이 네트워크는 다양한 영상을 대표하는 다양한 특징을 학습했습니다. 

**VGG-16모델**을 활용하여 Transfer Learning 을 진행합니다.

In [ ]:
from IPython.display import Image
Image('https://s3.ap-south-1.amazonaws.com/techleer/309.jpg')

### include_top 은 실제 vgg16의 마지막 부분인 분류기 (softmax를 포함한 1000개 category로 분류)를 포함하냐(True), 마느냐(False)를 선택합니다. 우리는 cat 이냐 dog 이냐를 분류하려고 하므로 False로 합니다.

In [ ]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

### 불러오는 vgg16은 새로 학습할 것이 아니므로 trainable = False로 합니다.

In [ ]:
transfer_model.trainable=False

In [ ]:
model_tr = Sequential([
    transfer_model, 
    Flatten(), 
    Dense(64, activation='relu'), 
    Dense(1, activation='sigmoid')
    ])

In [ ]:
model_tr.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

checkpoint_path = 'checkpoint.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path, 
        save_best_only=True, 
        save_weights_only=True, 
        monitor='val_loss', 
        verbose=1)

In [ ]:
transfer_history = model_tr.fit(train_data, 
                             validation_data=(valid_data),
                             steps_per_epoch=steps_per_epoch,
                             validation_steps=validation_steps,
                             epochs=EPOCHS, 
                             callbacks=[checkpoint, early_stopping])
model_tr.load_weights(checkpoint_path)                      
model_tr.save("cats-dogs-tr.h5")      

## 성능 Visualization

**Losses** 와 **Accuracy**를 보면, Transfer Learning 모델이 훨씬 초기 수렴이 빠를 뿐더러, 최종 성능도 accuracy 기준 **약 10%** 가량 우수하다는 것을 확인할 수 있습니다.

In [ ]:
EPOCHS = 19
plt.figure(figsize=(12, 10))
plt.plot(np.arange(1, EPOCHS+1), cnn_history.history['val_loss'], label='CNN')
plt.plot(np.arange(1, EPOCHS+1), transfer_history.history['val_loss'], label='TRANSFER')
plt.xlabel('EPOCHS')
plt.ylabel('LOSSES')
plt.title('LOSSES')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
plt.plot(np.arange(1, EPOCHS+1), cnn_history.history['val_accuracy'], label='CNN')
plt.plot(np.arange(1, EPOCHS+1), transfer_history.history['val_accuracy'], label='TRANSFER')
plt.xlabel('EPOCHS')
plt.ylabel('Accuracy')
plt.title('Accuracy Graph')
plt.legend()
plt.show()